In [1]:
import os
import xarray as xr
import rioxarray as rxr
from shapely.geometry import mapping
import geopandas as gpd
from rasterio.enums import Resampling
from datetime import datetime, timedelta
from calendar import monthrange

C:\Users\enhi\AppData\Local\Temp\ipykernel_12092\3827688388.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [19]:
main_dir =r'C:\Users\enhi\OneDrive - gmv.com\Documents\Projects\ANIN\Generating Indices\VCI\NDVI-16\Trial\Tiles\\'
Monthly_Copernicus = os.path.join(main_dir, 'Copernicus_monthly')
Max =os.path.join(main_dir, 'MAX')
Min =os.path.join(main_dir, 'MIN')
LC_path= os.path.join(main_dir, 'LC_300m.tif')
LCSentinel3_path= os.path.join(main_dir, 'LC_South Africa300.tif')

In [3]:
Copernicus_years = sorted(os.listdir(Monthly_Copernicus))
years_copernicus = {}
for year in Copernicus_years:
    months_copernicus={}
    for month in sorted(os.listdir(Monthly_Copernicus+ '\\' + year)):
        path = os.path.join(Monthly_Copernicus, year, month)
        image = rxr.open_rasterio(path, masked = True)
        months_copernicus[month[4:-4]]=image
    years_copernicus[year] = months_copernicus


In [4]:
NDVI_max= {}
NDVI_min= {}
Max_NDVI = sorted(os.listdir(Max))
Min_NDVI = sorted(os.listdir(Min))
for month in Max_NDVI:
    path = os.path.join(Max, month)
    image = rxr.open_rasterio(path, masked = True)
    NDVI_max[month[:-8]]=image
for month in Min_NDVI:
    path = os.path.join(Min, month)
    image = rxr.open_rasterio(path, masked = True)
    NDVI_min[month[:-8]]=image

    
    

In [22]:
Land_cover_CCI = rxr.open_rasterio(LC_path, masked = True)
Land_cover_S3 = rxr.open_rasterio(LCSentinel3_path, masked = True)
Land_cover_S3 = Land_cover_S3.rio.reproject_match(image)
for year in years_copernicus:
    #loop through each month in the year
    for month in years_copernicus[year]:
        #Assign the monthly NDVI data which matchs the year and month
        Monthly_NDVI = years_copernicus[year][month]
        #Taking the max NDVI which matches the month 
        Max_NDVI = NDVI_max[month]
        #Taking the min NDVI which matches the month
        Min_NDVI = NDVI_min[month]
        #Calculation of Vegetation Condition Index (VCI)
        VCI = (Monthly_NDVI - Min_NDVI)/(Max_NDVI - Min_NDVI)
        #add LC
        VCI_mask= xr.concat([VCI,Land_cover_S3, Land_cover_CCI],dim ='band')
        VCI_mask.rio.to_raster(main_dir + '\\' 'VCI_LC' + '\\'+ 'All'+'\\'+ month+ year+ '_VCI'+'.tif')
        #VCI_vegetation 
        VCI_veg = VCI_mask.where( ((VCI_mask[1]>=11) & (VCI_mask[1]<=170)) | ((VCI_mask[2]>=1)& (VCI_mask[2]<6)))
        VCI_veg.rio.to_raster(main_dir + '\\' 'VCI_LC' + '\\'+ 'Veg'+'\\'+ month+ year+ '_VCI'+'.tif')
        #VCI_cropland
        VCI_crop = VCI_mask.where( ((VCI_mask[1]>=11) & (VCI_mask[1]<=30)) | (VCI_mask[2]==4))
        VCI_crop.rio.to_raster(main_dir + '\\' 'VCI_LC' + '\\'+ 'Cropland'+'\\'+ month+ year+ '_VCI'+'.tif')
        #Export the VCI as tiff file
        
print('VCI files are created')

VCI files are created
